Proyecto Final
==============

### Tratamiento de Datos
### Máster de Ing. de Telecomunicación

## Autores

Juan Manuel Espinosa Moral ([100406523@alumnos.uc3m.es](mailto:100406523@alumnos.uc3m.es))

José Manuel García Núñez ([100544621@alumnos.uc3m.es](mailto:100544621@alumnos.uc3m.es))

In [ ]:
# Integración en Collab

# Librerías de drive
from google.colab import drive
import os, sys

# Montaje
drive.mount("/content/drive")

# Directorio actual
print(os.getcwd())

# Cambio de directorio al compartido
directory_path = "/content/drive/MyDrive/Colab Notebooks/proyecto_td/"  # path
# If para crear el directorio en su path en caso de no existir
if not os.path.exists(directory_path):
  os.makedirs(directory_path)
  print(f"Directory created: {directory_path}")

os.chdir(directory_path) # switch de directorio

## 1. Análisis de Variables de Entrada

- Carga del dataset: datos del archivo JSON.
- Categorías: las más frecuentes.
- Rating y visualizaciones.
- Análisis de correlación: categorias y variables de salida.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar el JSON
if not os.path.exists("full_format_recipes.json"):
  df = pd.read_json("data/full_format_recipes.json")
else:
  df = pd.read_json("full_format_recipes.json")

# Explorar categories en cuanto a aparición
category_counts = df['categories'].explode().value_counts()
print("Categorías más frecuentes:\n", category_counts.head())

# Top 10 categories
category_counts.head(10).plot(kind='bar', title="Frecuencia de las categorías")
plt.ylabel("Número de recetas")
plt.show()

# Analizar la relación entre categorías y ratings, filtrando por las más frecuentes
df_exploded = df.explode('categories')  # Expandir listas de categorías
df_exploded = df_exploded.reset_index(drop=True)  # Resetear el índice

# Filtrar por las 10 categorías con mejor rating
top_categories = category_counts.head(10).index.tolist()
df_filtered = df_exploded[df_exploded['categories'].isin(top_categories)]

plt.figure(figsize=(12, 6))
sns.boxplot(x='categories', y='rating', data=df_filtered)
plt.xticks(rotation=90)
plt.title("Distribución de ratings por categoría (Top 10)")
plt.show()

In [ ]:
# correlacion y heatmap de variables
correlation = df[['fat', 'protein', 'calories', 'sodium', 'rating']].corr()

print(correlation)

sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:


# Relación de ratings por categories
df_exploded = df.explode('categories')  # Expandir listas de categorías
df_exploded = df_exploded.reset_index(drop=True)
plt.figure(figsize=(12, 6))
sns.boxplot(x='categories', y='rating', data=df_exploded)
plt.xticks(rotation=90)
plt.title("Distribución de ratings por categoría")
plt.show()


In [ ]:
# import matplotlib.pyplot as plt

# Analisis logarítmico relacionando fat y rating
plt.scatter(df['fat'], df['rating'])
plt.xscale('log')  # Apply logarithmic scale to x-axis
plt.xlabel('Fat (grams) - Log Scale')
plt.ylabel('Rating')
plt.title('Fat vs. Rating (Log Scale)')
plt.show()

In [ ]:
# Relación entre rating y calories

# se definen los tramos de calorias
bins = [0, 200, 400, 600, 800, 1000, float('inf')]
labels = ['0-200', '201-400', '401-600', '601-800', '801-1000', '1001+']

# creación de una columna independiente con calorías
df['calorie_bins'] = pd.cut(df['calories'], bins=bins, labels=labels)

# creación de una medía de calorias en baase al rating
average_ratings = df.groupby('calorie_bins')['rating'].mean()

# graficación de resultados
plt.bar(average_ratings.index, average_ratings.values)
plt.xlabel('Calorie Range')
plt.ylabel('Average Rating')
plt.title('Average Rating vs. Calories')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# se definen los tramos de grasa (gramos)
bins_fat = [0, 10, 20, 30, 40, 50, float('inf')]
labels_fat = ['0-10', '11-20', '21-30', '31-40', '41-50', '51+']

# creación de una columna independiente con grasas
df['fat_bins'] = pd.cut(df['fat'], bins=bins_fat, labels=labels_fat)

# creación de una media de calorias en base a la cantidad de grasas
average_ratings = df.groupby('fat_bins')['rating'].mean()

# graficación de resultados
plt.bar(average_ratings.index, average_ratings.values)
plt.xlabel('Fat Range')
plt.ylabel('Average Rating')
plt.title('Average Rating vs. Fat')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# se definen los tramos de proteina (gramos)
bins_protein = [0, 10, 20, 30, 40, 50, float('inf')]
labels_protein = ['0-10', '11-20', '21-30', '31-40', '41-50', '51+']

# creación de una columna independiente con proteina
df['protein_bins'] = pd.cut(df['protein'], bins=bins_protein, labels=labels_protein)

# creación de una media de calorias en base a la cantidad de proteina
average_ratings = df.groupby('protein_bins')['rating'].mean()

# graficación de resultados
plt.bar(average_ratings.index, average_ratings.values)
plt.xlabel('Protein Range')
plt.ylabel('Average Rating')
plt.title('Average Rating vs. Protein')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Esta parte no se si tiene mucho sentido mantener

top_20_categories = df_exploded['categories'].value_counts().head(20).index.tolist()
df_filtered = df_exploded[df_exploded['categories'].isin(top_20_categories)]

variables = ['fat', 'protein', 'calories', 'sodium', 'rating']
for variable in variables:
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='categories', y=variable, data=df_filtered)
    plt.xticks(rotation=90)
    plt.title(f'{variable.capitalize()} Distribution by Top 20 Categories')
    plt.tight_layout()
    plt.show()

## 2. Implementación de un *pipeline* para el preprocesado de los textos

In [ ]:
import re
import spacy

# cargado de spaCy
nlp = spacy.load("en_core_web_sm")


# definición del método de procesado de texto del pipeline
def preprocess_text(text, use_spacy=False):

    # Si el texto es NaN o no es string/list, convertirlo a cadena vacía
    if not isinstance(text, (str, list)):
        text = ''

    # si el texto es una lista, unir sus elementos en una sola cadena
    if isinstance(text, list):
        text = ' '.join(text)

    # pasar a minúsculas
    text = text.lower()

    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-z\s]', '', text)

    if use_spacy:
        # Usar SpaCy para tokenización y lematización
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc if not token.is_stop]
    else:
        # Tokenizar el texto
        tokens = nltk.word_tokenize(text)

        # Eliminar stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]

        # Lematizar
        lemmatizer = nltk.WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Unir tokens de nuevo en un solo string
    processed_text = ' '.join(tokens)
    return processed_text

# Manejar valores NaN en la columna 'categories'
df['categories'] = df['categories'].fillna('')

# Aplicar el pipeline al conjunto de datos
df['processed_categories'] = df['categories'].apply(preprocess_text, use_spacy=True)

# print de resultados
print(df[['categories', 'processed_categories']].head())

# Repetir el proceso con desc
df['desc'] = df['desc'].fillna('')
df['processed_desc'] = df['desc'].apply(preprocess_text, use_spacy=True)
print(df[['desc', 'processed_desc']].head())



## 3. Representación vectorial de los documentos mediante tres procedimientos diferentes

### Procedimiento 1: TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Creación del objeto a vectorizar
vectorizer = TfidfVectorizer()

# conversión de datos para vectorizar
tfidf_matrix1 = vectorizer.fit_transform(df['processed_categories'])
# extracción de los nombres
feature_names1 = vectorizer.get_feature_names_out()
# creación del dataframe con los datos convertidos a arrays pasados por el método y los nombres extraidos
tfidf_df1 = pd.DataFrame(tfidf_matrix1.toarray(), columns=feature_names1, index=df.index)

# ------------------------------------------------------------------------------
# Repetición del proceso anterior para desc en lugar de categories
vectorizer2 = TfidfVectorizer()  # Or use: vectorizer = TfidfVectorizer() if reusing
tfidf_matrix2 = vectorizer2.fit_transform(df['processed_desc'])  # Or use: vectorizer.fit_transform if reusing
feature_names2 = vectorizer2.get_feature_names_out()  # Or use: vectorizer.get_feature_names_out() if reusing
tfidf_df2 = pd.DataFrame(tfidf_matrix2.toarray(), columns=feature_names2, index=df.index)

# print de los resultados
print("TF-IDF DataFrame for 'processed_categories':")
print(tfidf_df1)

print("\nTF-IDF DataFrame for 'processed_desc':")
print(tfidf_df2)

### Procedimiento 2: *Word2Vec*

In [ ]:
!pip install nltk==3.8.1
!pip install gensim

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import numpy as np
from gensim.models import Word2Vec


# Preparado de datos de texto
sentences = [row.split() for row in df['processed_categories']]

# Entrenamiento del modelo con las condiciones normales

model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

# 3. Obtén los vectores de palabras y promediados

def get_category_vector(category_words):
  """Obtiene el vector promedio para una categoría."""
  vectors = [model.wv[word] for word in category_words if word in model.wv]
  if vectors:
    return np.mean(vectors, axis=0)
  else:
    return np.zeros(model.vector_size)  # Vector de ceros si no hay palabras en el vocabulario

df['category_vector'] = df['processed_categories'].apply(lambda x: get_category_vector(x.split()))

# Muestra los primeros 5 elementos del vector para las primeras 5 filas
for index in range(5):
  print(f"Fila {index}: {df['category_vector'][index][:5]}")



In [ ]:
!pip install nltk==3.8.1
!pip install gensim

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import numpy as np
from gensim.models import Word2Vec

# 1. Preparar datos de texto para 'categories'
sentences1 = [row.split() for row in df['processed_categories']]

# 2. Entrenar el modelo para 'categories'
model1 = Word2Vec(sentences1, vector_size=100, window=5, min_count=5, workers=4)

# 3. Obtener vectores promediados para 'categories'
def get_vector(text_words, model):
  """Obtiene el vector promedio para un texto."""
  vectors = [model.wv[word] for word in text_words if word in model.wv]
  if vectors:
    return np.mean(vectors, axis=0)
  else:
    return np.zeros(model.vector_size)  # Vector de ceros si no hay palabras en el vocabulario

df['category_vector'] = df['processed_categories'].apply(lambda x: get_vector(x.split(), model1))

# Muestra de 5 resultados
print("Resultados para 'categories':")
for index in range(5):
  print(f"Fila {index}: {df['category_vector'][index][:5]}")

# ------------------------------------------------------------------------------
# Repetir el proceso para desc
sentences2 = [row.split() for row in df['processed_desc']]
model2 = Word2Vec(sentences2, vector_size=100, window=5, min_count=5, workers=4)
df['desc_vector'] = df['processed_desc'].apply(lambda x: get_vector(x.split(), model2))


print("\nResultados para 'desc':")
for index in range(5):
  print(f"Fila {index}: {df['desc_vector'][index][:5]}")

### **DEBUG**: *Testing* de los metodos de vectorización TF-IDF y vec2sec

In [ ]:
from wordcloud import WordCloud
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA # Import PCA
import numpy as np # Import numpy

# Visualización de TF-IDF con Nube de Palabras
word_weights = dict(zip(feature_names1, tfidf_matrix1.sum(axis=0).tolist()[0]))
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_weights)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Nube de Palabras TF-IDF")
plt.show()

# Visualización de Word2Vec con PCA
pca = PCA(n_components=2)
vectors_2d = pca.fit_transform(df['category_vector'].to_list())
plt.figure(figsize=(10, 10))
plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1])
plt.title("Visualización de vectores Word2Vec con PCA")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.show()

# Visualización de Word2Vec con t-SNE

tsne = TSNE(n_components=2, random_state=42)
# Convert the list of vectors to a 2D NumPy array
category_vectors = df['category_vector'].to_list()
vectors_2d = tsne.fit_transform(np.array(category_vectors).reshape(len(category_vectors), -1)) # Reshape if necessary
plt.figure(figsize=(10, 10))
plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1])
plt.title("Visualización de vectores Word2Vec con t-SNE")
plt.xlabel("Dimensión 1")
plt.ylabel("Dimensión 2")
plt.show()

### Procedimiento 3: *Embeddings* contextuales calculados a partir de modelos basados en *transformers*

In [ ]:
# Procedimiento 3